In [ ]:
# Lab Assignment 4: Probabilistic Parsing with CYK Algorithm and Neural Dependency Parsing
# •	Implement the Cocke-Younger-Kasami (CYK) algorithm for parsing Context-Free Grammars (CFGs).
# •	Train a Neural Dependency Parser (e.g., using Stanza or spaCy) on a dataset like Universal Dependencies.
# •	Compare traditional parsing algorithms with neural parsing models in terms of accuracy and efficiency.

In [ ]:
# Step 1: Install Required Libraries
!pip install spacy nltk benepar --quiet
!python -m nltk.downloader punkt
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may r

In [ ]:
# Step 2: CYK Algorithm for Probabilistic Parsing
# Simple CFG in CNF format
grammar = {
    "S": [("NP", "VP")],
    "VP": [("V", "NP")],
    "NP": [("Det", "N")],
    "Det": [("the",)],
    "N": [("cat",), ("dog",)],
    "V": [("chased",), ("saw",)],
}

# Sentence to parse
sentence = "the cat chased the dog".split()

In [ ]:
def cyk_parser(sentence, grammar):
    n = len(sentence)
    table = [[set() for _ in range(n)] for _ in range(n)]

    # Fill in the table for words (1-length substrings)
    for j in range(n):
        for lhs, rhs_list in grammar.items():
            for rhs in rhs_list:
                if len(rhs) == 1 and rhs[0] == sentence[j]:
                    table[j][j].add(lhs)

    # Fill in the rest of the table
    for span in range(2, n + 1):  # span length
        for start in range(n - span + 1):
            end = start + span - 1
            for split in range(start, end):
                left_cell = table[start][split]
                right_cell = table[split + 1][end]
                for lhs, rhs_list in grammar.items():
                    for rhs in rhs_list:
                        if len(rhs) == 2:
                            if rhs[0] in left_cell and rhs[1] in right_cell:
                                table[start][end].add(lhs)

    return table, "S" in table[0][n - 1]

# Run CYK parser
table, is_valid = cyk_parser(sentence, grammar)
print("CYK Parse Table:")
for row in table:
    print(row)

print("\nCYK Output:", "Valid parse" if is_valid else "Invalid sentence")

CYK Parse Table:
[{'Det'}, {'NP'}, set(), set(), {'S'}]
[set(), {'N'}, set(), set(), set()]
[set(), set(), {'V'}, set(), {'VP'}]
[set(), set(), set(), {'Det'}, {'NP'}]
[set(), set(), set(), set(), {'N'}]

CYK Output: Valid parse


In [ ]:
# Step 3: Neural Dependency Parsing using spaCy
import spacy
from spacy import displacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Parse the same sentence
doc = nlp("The cat chased the dog.")

# Display dependencies
print("\nDependency Parse Tree:")
for token in doc:
    print(f"{token.text:<10} {token.dep_:<10} {token.head.text:<10} POS: {token.pos_}")

# Visualize in Colab
displacy.render(doc, style='dep', jupyter=True, options={'compact': True})


Dependency Parse Tree:
The        det        cat        POS: DET
cat        nsubj      chased     POS: NOUN
chased     ROOT       chased     POS: VERB
the        det        dog        POS: DET
dog        dobj       chased     POS: NOUN
.          punct      chased     POS: PUNCT


In [ ]:
# Step 4: Compare CYK vs Neural Parsing
print("\n🔁 Comparison Summary")
print("-" * 40)
print("Approach           | Output")
print("-" * 40)
print(f"CYK Parser         | {'Valid parse' if is_valid else 'Invalid'}")
print("Neural Parser      | Dependency tree shown above")
print("\nCYK is rule-based, slow for large grammars.\nNeural models are faster, data-driven, and more scalable.")


🔁 Comparison Summary
----------------------------------------
Approach           | Output
----------------------------------------
CYK Parser         | Valid parse
Neural Parser      | Dependency tree shown above

CYK is rule-based, slow for large grammars.
Neural models are faster, data-driven, and more scalable.
